## TopTenParts | Highest Quantity Parts in a LEGO Set

#### This notebook will find the Star Wars LEGO set with the most pieces, and then find the 10 most common parts within that set. It uses LEGO set, part, and theme information from [Rebrickable](https://rebrickable.com/downloads/). The data is snapshots of multiple tables from the Rebrickable database downloaded in July 2022. Rebrickable updates their files daily. This notebook will read 5 .csv files, merge data, clean data, and create "top_ten_parts.csv" in the "CSVs" folder in this repo. This .csv file is used in a [Tableau Dashboard](https://public.tableau.com/app/profile/jared.sage/viz/BricksandPieces/BricksandPieces).

#### The results of this notebook will be an exact answer to the 10 most common parts.

#### Start by importing the necessary packages to run the notebook. Pandas will be used for importing, cleaning, and merging data. Requests will be used for API calls. PythonScripts is a folder of scripts made for this project. data_clean is for functions used across the 2 or more notebooks in this repo. 

In [ ]:
import pandas as pd 
import requests
import PythonScripts.data_clean as dc
from pathlib import Path

#### We are going to create several DataFrames based on the Rebrickable data. The database diagram on the [Rebrickable downloads](https://rebrickable.com/downloads/) page shows the relationship between the tables and will frame how we merge the data. We are going to create a DataFrame for:

- Inventory IDs/Set Numbers
- Inventory Parts - all the parts and part quantities within each inventory ID
- Sets - all the details for a set number
- Parts - all the details about a part, such as color
- Colors - all the details about a specific color

#### We are only going to read in the columns needed to reduce the amount of data we'll have to parse through. Output is the shape and first 3 rows of each DataFrame.

In [ ]:
# Read CSVs off local drive and convert them each to a DataFrame

inv_path = dc.csv_path('inventories.csv')
inv_df = pd.read_csv(inv_path, usecols=['id','set_num'])

inv_parts_path = dc.csv_path('inventory_parts.csv')
inv_parts_df = pd.read_csv(inv_parts_path, usecols=['inventory_id', 'part_num', 'color_id', 'quantity'])

sets_path = dc.csv_path('sets.csv')
set_df = pd.read_csv(sets_path, usecols=['set_num', 'name', 'year', 'theme_id', 'num_parts'])

parts_path = dc.csv_path('parts.csv')
parts_df = pd.read_csv(parts_path, usecols=['part_num', 'name'])

colors_path = dc.csv_path('colors.csv')
colors_df = pd.read_csv(colors_path, usecols=['id', 'name', 'rgb'])

print(f'inv_df shape: {inv_df.shape}')
print(inv_df.head(n=3))
print(f'\ninv_parts_df shape: {inv_parts_df.shape}')
print(inv_parts_df.head(n=3))
print(f'\nset_df shape: {set_df.shape}')
print(set_df.head(n=3))
print(f'\nparts_df shape: {parts_df.shape}')
print(parts_df.head(n=3))
print(f'\ncolors_df shape: {colors_df.shape}')
print(colors_df.head(n=3))

#### We will perform several merges to create one giant DataFrame. To make merges possible, we rename column labels across DataFrames so that Series with the same data point/value have the same column name. Output is the starting and new column labels for the inv_df, parts_df, and color_df.

In [ ]:
# Rename columns in inv_df and parts_df for easier joining
print('Original column labels:')
print(f'inv_df: {inv_df.columns}')
print(f'parts_df: {parts_df.columns}')
print(f'colors_df: {colors_df.columns}')

inv_rename_dict = {'id' : 'inventory_id',
                   'set_num' : 'set_num'}
inv_df.rename(columns=inv_rename_dict, inplace=True)

parts_rename_dict = {'part_num' : 'part_num',
                     'name' : 'part_name'}
parts_df.rename(columns=parts_rename_dict, inplace=True)

colors_rename_dict = {'id' : 'color_id',
                     'name' : 'color_name'}
colors_df.rename(columns=colors_rename_dict, inplace=True)

print('\nRenamed column labels:')
print(f'inv_df: {inv_df.columns}')
print(f'parts_df: {parts_df.columns}')
print(f'colors_df: {colors_df.columns}')

#### Now we merge. Starting point is the Inventory ID DataFrame. We then do 4 joins to add on data in the following order:
1. Inventory Parts - part_numbers, color, and quantities for each part
2. Sets - set names, themes, release year, and number of total parts in the set
3. Parts - the name of each part
4. Color - color name and hex color code (rgb column)

#### Output shows the increasing number of columns added on to the DataFrames.

In [ ]:
# Merge all DataFrames into one larger DataFrame with all data points
print(f'Original inv_df shape: {inv_df.shape}')

all_merged_df = inv_df.merge(inv_parts_df, how='inner', left_on='inventory_id', right_on='inventory_id')
print(f'Post-first merge shape: {all_merged_df.shape}')
      
all_merged_df = all_merged_df.merge(set_df, how='left', left_on='set_num', right_on='set_num')
print(f'Post-second merge shape: {all_merged_df.shape}')

all_merged_df = all_merged_df.merge(parts_df, how='inner', left_on='part_num', right_on='part_num')
print(f'Post-third all_merged_df shape: {all_merged_df.shape}')

all_merged_df = all_merged_df.merge(colors_df, how='inner', left_on='color_id', right_on='color_id')
print(f'current all_merged_df shape: {all_merged_df.shape}')



#### Our DataFrame currently has information for all LEGO sets. But we want to focus on Star Wars-themed sets. We reduce the DataFrame to sets that have one of the theme_ids relating to Star Wars. Output shows the theme IDs we wanted and the unique theme IDs after removing rows. They should be equal to verify we only have what we want. 

In [ ]:
# Remove all non-Star Wars themes from the Dataframe
sw_theme_ids = [18, 158, 171, 209, 261]
print(f'Star Wars Theme IDs: {sw_theme_ids}')
all_merged_df = all_merged_df[all_merged_df['theme_id'].isin(sw_theme_ids)]
unique_themes = all_merged_df['theme_id'].unique()

print(f'Unique values in theme_id column in DataFrame: {unique_themes}')

#### Now we want to figure out what the Star Wars set with the most pieces is. We group quantities of parts by set number and report the sums. Then we can find the set number with the highest sum. Output is the set number with the most parts.

In [ ]:
# Find the set number with the most pieces
piece_count = all_merged_df.groupby(['set_num'])['quantity'].sum()
max_count = piece_count.idxmax()
print(f'Set with highest count: {max_count}')

#### Next we make a DataFrame containing rows where the set number equals the set number with the most parts. Output shows the shape of the DataFrame, now containing only the 730 parts, and the first 3 rows of the DataFrame.

In [ ]:
# Drop all rows from the large DataFrame except the set with most pieces
print(f'Current all_merged_df shape: {all_merged_df.shape}')

mask = all_merged_df[all_merged_df['set_num'] != max_count].index
all_merged_df.drop(mask, inplace=True)

print(f'Final all_merged_df shape: {all_merged_df.shape}')
print(f'\n{all_merged_df.head(n=3)}')

#### Looking at the above rows we can see that the year, theme_id, and num_parts columns are currently floats. To assist with any calculations in Tableau and a cleaner look, convert those columns to int values. Output shows new dtypes for values in those columns.

In [ ]:
# Convert year, theme_id, and num_parts columns to integers
all_merged_df['year'] = all_merged_df['year'].astype(pd.Int64Dtype())
all_merged_df['theme_id'] = all_merged_df['theme_id'].astype(pd.Int64Dtype())
all_merged_df['num_parts'] = all_merged_df['num_parts'].astype(pd.Int64Dtype())
print('year dtype: ',type(all_merged_df['year'].iloc[1]))
print('theme_id dtype: ',type(all_merged_df['theme_id'].iloc[1]))
print('num_parts dtype: ',type(all_merged_df['num_parts'].iloc[1]))

#### Currently, the DataFrame has information for all 730 part numbers for the set in question. We only want to know info about the top 10 parts. We can find the 10 largest values in the quantity column and make a new DataFrame containing all information for rows that have those 10 largest values. Output is the new DataFrame.

In [ ]:
# Make a new DataFrame with just the top 10 parts with highest quantity.
top_ten_parts = all_merged_df['quantity'].nlargest(n=10, keep='first')
top_ten_df = all_merged_df[all_merged_df['quantity'].isin(top_ten_parts)]
top_ten_df.reset_index(drop=True, inplace=True)
top_ten_df

#### Data cleaning and merging is complete. We save that data as a .csv to use in Tableau. A file called "top_ten_parts.csv" will be created in the "CSVs" folder of this repo if you would like to review the final output. Output is the .csv file in the repo. 

In [ ]:
# Write a .csv of the parts on the local drive for use in Tableau visualization
file_path = dc.csv_path('top_ten_parts.csv')
top_ten_df.to_csv(file_path)